In [204]:
!pip install yfinance
import yfinance as yf   
import pandas as pd
import numpy as np

In [205]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC


In [206]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","BTC-USD","LTC-USD","TRX-USD","GGPI","BIM","AI","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD"]
symbols = ["AAPL"]

dfs = []
for i in symbols:
   data = yf.download(i,period="MAX",interval="1d")
   dfs.append(data)


[*********************100%***********************]  1 of 1 completed


In [207]:
for df in dfs:
  print(df.isnull().sum(),end=" ")
  print(df.shape)
  

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64 (10420, 6)


In [208]:
print(dfs[0])

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1980-12-12    0.128348    0.128906    0.128348    0.128348    0.100323   
1980-12-15    0.122210    0.122210    0.121652    0.121652    0.095089   
1980-12-16    0.113281    0.113281    0.112723    0.112723    0.088110   
1980-12-17    0.115513    0.116071    0.115513    0.115513    0.090291   
1980-12-18    0.118862    0.119420    0.118862    0.118862    0.092908   
...                ...         ...         ...         ...         ...   
2022-04-04  174.570007  178.490005  174.440002  178.440002  178.440002   
2022-04-05  177.500000  178.300003  174.419998  175.059998  175.059998   
2022-04-06  172.360001  173.630005  170.130005  171.830002  171.830002   
2022-04-07  171.160004  173.360001  169.850006  172.139999  172.139999   
2022-04-08  171.779999  171.767197  169.220001  171.250000  171.250000   

               Volume  
Date         

In [225]:
fixed_dfs = []
for df in dfs:
    full = []
    df = np.array(df)
    for i in range(15, df.shape[0]-1):
        fifteen_days_ago = i-15
        fourteen_days_ago = i-14
        thirteen_days_ago = i-13
        twelve_days_ago = i-12
        eleven_days_ago = i-11
        ten_days_ago = i-10
        nine_days_ago = i-9
        eight_days_ago = i-8
        seven_days_ago = i-7
        six_days_ago = i-6
        five_days_ago = i-5
        four_days_ago = i-4
        three_days_ago = i-3
        two_days_ago = i - 2
        yesterday = i - 1
        today = i
        tomorrow = i + 1
        if df[tomorrow][3] > df[today][3]:
            future = 1
        else:
            future = 0


        new_lst = [df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],df[twelve_days_ago][3],df[eleven_days_ago][3],
                   df[ten_days_ago][3], df[nine_days_ago][3], df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3],
                   df[five_days_ago][3], df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                   df[today][3], future]

        full.append(new_lst)


    fuu = np.array([full])
    lst = []
    for i in range(0, fuu.shape[1]):
        lst.append(fuu[0][i])
    
    mm = np.array(lst)


    column_names = ["fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today", "result"]

    dff = pd.DataFrame(mm, columns=column_names)
    
    fixed_dfs.append(dff)

In [210]:
for i in fixed_dfs:
  print(i.shape)

(10404, 17)


In [211]:
data = fixed_dfs[0]
for i in range(1,len(symbols)):
  data = pd.concat([data,fixed_dfs[i]], ignore_index = True)

In [212]:
data

,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,ten_days_ago,nine_days_ago,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,0.128348,0.121652,0.112723,0.115513,0.118862,0.126116,0.132254,0.137835,0.145089,0.158482,0.160714,0.156808,0.152344,0.154018,0.150670,0.143973,0.0
1,0.121652,0.112723,0.115513,0.118862,0.126116,0.132254,0.137835,0.145089,0.158482,0.160714,0.156808,0.152344,0.154018,0.150670,0.143973,0.137835,0.0
2,0.112723,0.115513,0.118862,0.126116,0.132254,0.137835,0.145089,0.158482,0.160714,0.156808,0.152344,0.154018,0.150670,0.143973,0.137835,0.135045,1.0
3,0.115513,0.118862,0.126116,0.132254,0.137835,0.145089,0.158482,0.160714,0.156808,0.152344,0.154018,0.150670,0.143973,0.137835,0.135045,0.142299,0.0
4,0.118862,0.126116,0.132254,0.137835,0.145089,0.158482,0.160714,0.156808,0.152344,0.154018,0.150670,0.143973,0.137835,0.135045,0.142299,0.141183,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10399,154.729996,150.619995,155.089996,159.589996,160.619995,163.979996,165.380005,168.820007,170.210007,174.070007,174.720001,175.600006,178.960007,177.770004,174.610001,174.309998,1.0
10400,150.619995,155.089996,159.589996,160.619995,163.979996,165.380005,168.820007,170.210007,174.070007,174.720001,175.600006,178.960007,177.770004,174.610001,174.309998,178.440002,0.0
10401,155.089996,159.589996,160.619995,163.979996,165.380005,168.820007,170.210007,174.070007,174.720001,175.600006,178.960007,177.770004,174.610001,174.309998,178.440002,175.059998,0.0
10402,159.589996,160.619995,163.979996,165.380005,168.820007,170.210007,174.070007,174.720001,175.600006,178.960007,177.770004,174.610001,174.309998,178.440002,175.059998,171.830002,1.0


In [213]:
data = data.dropna()

In [214]:
X = data.drop("result",axis=1)
xTrain, xTest, yTrain, yTest = train_test_split(X, data["result"], test_size=0.1,random_state=11)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)


(9363, 16)
(9363,)
(1041, 16)
(1041,)


In [215]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
clf.score(xTest,yTest)

0.5360230547550432

In [216]:
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
logisticRegr.score(xTest, yTest)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.5206532180595581

In [217]:
svm = SVC()
svm.fit(xTrain,yTrain)
svm.score(xTest,yTest)

0.5446685878962536

In [218]:
from sklearn.neighbors import NearestCentroid
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
nc.score(xTest,yTest)

0.5302593659942363

In [219]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xTrain, yTrain).score(xTest,yTest)

0.5244956772334294

In [220]:
from sklearn import tree
tree = tree.DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
tree.score(xTest,yTest)

0.5225744476464937

In [221]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.score(xTest, yTest)

0.5341018251681076

In [222]:
from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier(n_neighbors=7)
clf2.fit(xTrain, yTrain)
clf2.score(xTest,yTest)

0.49279538904899134

In [223]:
model = Sequential()
model.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(16,)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 64)                1088      
                                                                 
 dropout_36 (Dropout)        (None, 64)                0         
                                                                 
 dense_49 (Dense)            (None, 64)                4160      
                                                                 
 dropout_37 (Dropout)        (None, 64)                0         
                                                                 
 dense_50 (Dense)            (None, 32)                2080      
                                                                 
 dropout_38 (Dropout)        (None, 32)                0         
                                                                 
 dense_51 (Dense)            (None, 1)               

In [224]:
model.fit(xTrain,yTrain,epochs=15,batch_size=32,validation_data=(xTest,yTest))

Epoch 1/15
293/293 [==============================] - 4s 8ms/step - loss: 2.3522 - accuracy: 0.5057 - val_loss: 0.6978 - val_accuracy: 0.5120
Epoch 2/15
293/293 [==============================] - 2s 6ms/step - loss: 0.8565 - accuracy: 0.5083 - val_loss: 0.6930 - val_accuracy: 0.5120
Epoch 3/15
293/293 [==============================] - 2s 7ms/step - loss: 0.7434 - accuracy: 0.5087 - val_loss: 0.6930 - val_accuracy: 0.5120
Epoch 4/15
293/293 [==============================] - 1s 4ms/step - loss: 0.7023 - accuracy: 0.5052 - val_loss: 0.6930 - val_accuracy: 0.5120
Epoch 5/15
293/293 [==============================] - 2s 5ms/step - loss: 0.7063 - accuracy: 0.5081 - val_loss: 0.6930 - val_accuracy: 0.5120
Epoch 6/15
293/293 [==============================] - 2s 6ms/step - loss: 0.7001 - accuracy: 0.5065 - val_loss: 0.6929 - val_accuracy: 0.5120
Epoch 7/15
293/293 [==============================] - 1s 4ms/step - loss: 0.6988 - accuracy: 0.5081 - val_loss: 0.6929 - val_accuracy: 0.5120
Epoch 